<a href="https://colab.research.google.com/github/Sadman26/FUZZY/blob/master/InceptionV3_Strawberry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [4]:
train_dir = '/content/drive/MyDrive/strawberry/sl_train'
validation_dir = '/content/drive/MyDrive/strawberry/sl_test'

In [5]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [6]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)


In [7]:
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=IMG_SIZE,
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              target_size=IMG_SIZE,
                                                              batch_size=BATCH_SIZE,
                                                              class_mode='categorical')

Found 775 images belonging to 3 classes.
Found 689 images belonging to 3 classes.


In [8]:
# Load the InceptionV3 model and freeze its layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
for layer in base_model.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 1s 0us/step


In [12]:
# Add a custom classifier on top of the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [13]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
# Train the model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=validation_generator)

Epoch 1/10
25/25 [==============================] - 263s 10s/step - loss: 0.7173 - accuracy: 0.7535 - val_loss: 0.1937 - val_accuracy: 0.9289
Epoch 2/10
25/25 [==============================] - 260s 11s/step - loss: 0.1620 - accuracy: 0.9394 - val_loss: 0.0955 - val_accuracy: 0.9768
Epoch 3/10
25/25 [==============================] - 253s 10s/step - loss: 0.1229 - accuracy: 0.9548 - val_loss: 0.0757 - val_accuracy: 0.9826
Epoch 4/10
25/25 [==============================] - 256s 10s/step - loss: 0.0942 - accuracy: 0.9690 - val_loss: 0.0617 - val_accuracy: 0.9884
Epoch 5/10
25/25 [==============================] - 203s 8s/step - loss: 0.0742 - accuracy: 0.9703 - val_loss: 0.0585 - val_accuracy: 0.9855
Epoch 6/10
25/25 [==============================] - 257s 10s/step - loss: 0.0589 - accuracy: 0.9832 - val_loss: 0.0509 - val_accuracy: 0.9898
Epoch 7/10
25/25 [==============================] - 256s 10s/step - loss: 0.0603 - accuracy: 0.9819 - val_loss: 0.0494 - val_accuracy: 0.9840
Epoch 8

In [15]:
loss, accuracy = model.evaluate(validation_generator)
print('Validation accuracy:', accuracy)

22/22 [==============================] - 91s 4s/step - loss: 0.0316 - accuracy: 0.9942
Validation accuracy: 0.994194507598877
